In [ ]:
# File: friction.ipynb
# Code: Claude Code and Codex
# Review: Ryoichi Ando (ryoichi.ando@zozo.com)
# License: Apache v2.0

In [ ]:
import numpy as np
from frontend import App

# create an app
app = App.create("friction")

# create an armadillo tetrahedral mesh
V, F, T = app.mesh.preset("armadillo").decimate(2400).tetrahedralize().normalize()
app.asset.add.tet("armadillo", V, F, T)

# create a rectangular slope mesh
V, F = app.mesh.rectangle(res_x=43, width=10, height=3, ex=[1, 0, 0], ey=[0, 0, 1])
app.asset.add.tri("slope", V, F)

# create a scene
scene = app.scene.create()

# add armadillo with friction properties
armadillo = scene.add("armadillo")
armadillo.param.set("poiss-rat", 0.49).set("friction", 0.51)
armadillo.rotate(180, "y").rotate(-90, "x").rotate(-30, "z").at(
    -3.8, 2.35, -0.25
).jitter()

# add sloped surface with angle based on friction coefficient
deg = 180 * np.arctan(0.5) / np.pi
scene.add("slope").rotate(-deg, "z").pin()

# compile the scene and report stats
scene = scene.build().report()

# preview the initial scene
scene.preview()

In [ ]:
# create a new session with the compiled scene
session = app.session.create(scene)

# set session parameters
(
    session.param.set("dt", 0.01)
    .set("min-newton-steps", 32)
    .set("frames", 700)
)

# build this session
session = session.build()

In [ ]:
# start the simulation and live-preview the results
session.start().preview()

# also show simulation logs in realtime
session.stream()

In [ ]:
# create an animation from the simulation results
session.animate()

In [ ]:
# export the animation to file
session.export.animation()

In [ ]:
# this is for CI
if app.ci:
    assert session.finished()